In [1]:
import numpy as np
import random 
from scipy.stats import unitary_group
# defining Pauli matrices
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])

# defining rotation gates
def Ry(A):
    M = np.array([[np.cos(A/2),-np.sin(A/2)],[np.sin(A/2), np.cos(A/2)]])
    return M
def Rz(A):
    M = np.array([[np.exp(-1j*A/2),0],[0,np.exp(1j*A/2)]])
    return M
    
# Parameterising the general unitary with rotation gates
def U (A0,A1,A2): # Arguments: Euler Angles
    M = Rz(A1)@Ry(A0)@Rz(A2)
    return M

# derivatives wrt A0,A1,A2
def DA0(A0,A1,A2):
    dA0 = (-1j/2)*(Rz(A1)@Y@Ry(A0)@Rz(A2))
    return dA0
def DA1(A0,A1,A2):
    dA1 = (-1j/2)*(Z@Rz(A1)@Ry(A0)@Rz(A2))
    return dA1
def DA2 (A0,A1,A2):
    dA2 = (-1j/2)*(Rz(A1)@Ry(A0)@Z@Rz(A2))
    return dA2

# generate random unitary 
def Urand():
    A0=random.uniform(0.0, 4*np.pi)
    A1=random.uniform(0.0, 4*np.pi)
    A2=random.uniform(0.0, 4*np.pi)
    Urand= U(A0,A1,A2)
    return Urand

# defining inverse
def inverse(M):
    M_inverse = M.transpose().conjugate()
    return M_inverse



In [2]:
# function to define RHS for each qubit
def Utbm(j, A, U_goal):
    U1 = U(A[0,0],A[0,1],A[0,2])
    U2 = U(A[1,0],A[1,1],A[1,2])
    U3 = U(A[2,0],A[2,1],A[2,2])
    U4 = U(A[3,0],A[3,1],A[3,2])
    U5 = U(A[4,0],A[4,1],A[4,2])
    U6 = U(A[5,0],A[5,1],A[5,2])
    U7 = U(A[6,0],A[6,1],A[6,2])
    U8 = U(A[7,0],A[7,1],A[7,2])
    Ulist = [U1,U2,U3,U4,U5,U6,U7,U8]
    Ulist[j-1] = np.eye(2)
    U12 = np.kron(Ulist[0],Ulist[1])
    U34 = np.kron(Ulist[2],Ulist[3])
    U56 = np.kron(Ulist[4],Ulist[5])
    U78 = np.kron(Ulist[6],Ulist[7])
    if j==1:
        Uf = np.eye(4)
        Ub = U12@CNOT@U34@CNOT@U56@CNOT@U78
    elif j==2:
        Uf = U12
        Ub = CNOT@U34@CNOT@U56@CNOT@U78
    elif j==3:
        Uf = U12@CNOT
        Ub =U34@CNOT@U56@CNOT@U78
    elif j==4:
        Uf = U12@CNOT@U34
        Ub =CNOT@U56@CNOT@U78
    elif j==5:
        Uf = U12@CNOT@U34@CNOT
        Ub = U56@CNOT@U78
    elif j==6:
        Uf = U12@CNOT@U34@CNOT@U56
        Ub = CNOT@U78
    elif j==7:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT
        Ub = U78
    elif j==8:
        Uf = U12@CNOT@U34@CNOT@U56@CNOT@U78
        Ub = np.eye(4)
    U_tbm = inverse(Uf)@U_goal@inverse(Ub)
    return U_tbm

In [3]:
def gradient_descent(A, iterations,alpha,U_goal):
    Aopt = np.copy(A)
    for j in range(1,9):
        # rhs of the optimization
        Ui = Utbm(j,Aopt,U_goal)
        A0=Aopt[j-1][0]
        A1=Aopt[j-1][1]
        A2=Aopt[j-1][2]
        #print("initial:",A0,A1,A2)

        # defining LHS 
        U_initial= U(A0,A1,A2)
        if j%2!=0:
            Uxyz = np.kron(U_initial,np.eye(2))
            print(j)
            print(Uxyz)
        else:
            Uxyz = np.kron(np.eye(2), U_initial)
            print(j)
            print(Uxyz)
            
        print(Ui@inverse(Uxyz))
        Tr = np.trace(Ui@inverse(Uxyz))
        print(Tr)
        Abs = np.abs(Tr)
        print(Abs)
        Uerr = 4 - Abs
        print(Uerr)
        for i in range(iterations):
            Tr = np.trace(Ui@inverse(Uxyz))
            Abs = np.abs(Tr)
            Uerr = 4 - Abs
            #print(Abs/Tr)
            #print(Uerr)
            if Uerr <= 1e-8:
                break
            else:
                dA0 = DA0(A0,A1,A2)
                dA1 = DA1(A0,A1,A2)
                dA2 = DA2(A0,A1,A2)
                if j%2!=0:
                    UdA0 = np.kron(dA0,np.eye(2))
                    UdA1 = np.kron(dA1,np.eye(2))
                    UdA2 = np.kron(dA2,np.eye(2))
                else:
                    UdA0 = np.kron(np.eye(2),dA0)
                    UdA1 = np.kron(np.eye(2),dA1)
                    UdA2 = np.kron(np.eye(2),dA2)
                
                Grad_A0 = -np.conj((Abs/Tr))*(np.trace(Ui@inverse(UdA0)))
                Grad_A0R = Grad_A0.real
                Grad_A1 = -np.conj((Abs/Tr))*(np.trace(Ui@inverse(UdA1)))
                Grad_A1R = Grad_A1.real
                Grad_A2 = -np.conj((Abs/Tr))*(np.trace(Ui@inverse(UdA2)))
                Grad_A2R = Grad_A2.real
                
                A0 = A0 - alpha*Grad_A0R
                A1 = A1 - alpha*Grad_A1R
                A2 = A2 - alpha*Grad_A2R
                #print(Grad_A0, Grad_A1, Grad_A2)
                if j%2 != 0:
                    Uxyz = np.kron(U(A0,A1,A2),np.eye(2))
                else:
                    Uxyz = np.kron(np.eye(2), U(A0,A1,A2) )
            #print(Uxyz)
        #print(Aopt)
    return Aopt

In [4]:
CNOT = [[1,0,0,0],[0,0,0,1],[0,0,1,0],[0,1,0,0]] # switch tensor control

# goal unitary
U_goal = unitary_group.rvs(4)

A = np.zeros((8,3))
for i in range(8):
    for j in range(3):
        A[i][j] = random.uniform(0.0, 4*np.pi) 


AOpt =np.copy(A)
#for i in range(100):
AOpt = gradient_descent(AOpt,1000,0.001,U_goal)

#print("AOpt =",AOpt)

#A_result = gradient_descent(U_tbm,1000,0.01)
#verification
U_12 = np.kron(U(AOpt[0][0], AOpt[0][1], AOpt[0][2]), U(AOpt[1][0], AOpt[1][1], AOpt[1][2]))
U_34 = np.kron(U(AOpt[2][0], AOpt[2][1], AOpt[2][2]), U(AOpt[3][0], AOpt[3][1], AOpt[3][2]))
U_56 = np.kron(U(AOpt[4][0], AOpt[4][1], AOpt[4][2]), U(AOpt[5][0], AOpt[5][1], AOpt[5][2]))
U_78 = np.kron(U(AOpt[6][0], AOpt[6][1], AOpt[6][2]), U(AOpt[7][0], AOpt[7][1], AOpt[7][2]))

U_found = U_12 @ CNOT @ U_34 @ CNOT @ U_56 @ CNOT @ U_78




1
[[ 0.44829524-0.57644335j  0.        +0.j         -0.59570571+0.3344834j
  -0.        +0.j        ]
 [ 0.        +0.j          0.44829524-0.57644335j -0.        +0.j
  -0.59570571+0.3344834j ]
 [ 0.59570571+0.3344834j   0.        +0.j          0.44829524+0.57644335j
   0.        +0.j        ]
 [ 0.        +0.j          0.59570571+0.3344834j   0.        +0.j
   0.44829524+0.57644335j]]
[[ 0.33403873-0.09032062j -0.69066896+0.00691593j  0.05647937-0.37869907j
   0.50620843-0.01841151j]
 [-0.1170513 +0.12070283j -0.01945373-0.06115297j  0.67518995+0.20242365j
   0.12463155-0.67470131j]
 [ 0.04791187+0.14789843j -0.59330234+0.32678897j  0.11706029+0.52626955j
  -0.43123433+0.20101246j]
 [-0.09774536-0.90458083j -0.08330723-0.23037282j  0.24863886+0.06368872j
  -0.18342176+0.11245688j]]
(0.24822352226449892+0.4872528419033898j)
0.5468365834034186
3.453163416596581
2
[[-0.26715883-0.14724109j  0.78453403+0.53986348j  0.        -0.j
   0.        +0.j        ]
 [-0.78453403+0.53986348j -0.26

1. Gradient descent loop - ill-defined, values reset after every 10 iterations.
2. Definition of gradient ?

1. How fast does it converge
2. What's a stable learning rate
3. Does it reliably converge
4. Compare different unitaries and their convergence rate

In [34]:
print(U_goal*inverse(U_found))

[[ 0.29509325-0.17571494j -0.14388745+0.03928156j  0.21673694-0.04357867j
  -0.19073235-0.08191437j]
 [-0.12655443+0.04519348j  0.0869161 -0.0588654j   0.20305086+0.26495593j
  -0.19052477-0.27348331j]
 [ 0.21407423-0.12002487j -0.16736425-0.40598044j  0.01490271-0.02887207j
   0.00805519-0.10079836j]
 [ 0.01518216+0.18981285j  0.06104235+0.25788621j -0.08518108+0.10266612j
   0.10318934-0.09052514j]]


In [ ]:
#verification
U_1 = U(AOpt[0][0],AOpt[0][1],AOpt[0][2])
U_2 = U(AOpt[1][0],AOpt[1][1],AOpt[1][2])
U_3 = U(AOpt[2][0],AOpt[2][1],AOpt[2][2])
U_4 = U(AOpt[3][0],AOpt[3][1],AOpt[3][2])
U_5 = U(AOpt[4][0],AOpt[4][1],AOpt[4][2])
U_6 = U(AOpt[5][0],AOpt[5][1],AOpt[5][2])
U_7 = U(AOpt[6][0],AOpt[6][1],AOpt[6][2])
U_8 = U(AOpt[7][0],AOpt[7][1],AOpt[7][2])


U_12 = np.kron(U_1,U_2)
U_34 = np.kron(U_3,U_4)
U_56 = np.kron(U_5,U_6)
U_78 = np.kron(U_7,U_8)
U_found = U_12@CNOT@U_34@CNOT@U_56@CNOT@U_78
